# Import Library

In [1]:
!pip install Sastrawi

In [2]:
import pandas as pd
import numpy as np

import re
import string

import nltk
nltk.download('punkt_tab')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

import csv
import requests
from io import StringIO

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from xgboost import XGBClassifier

from sklearn.metrics import accuracy_score

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Embedding, Dense, Flatten
from tensorflow.keras.callbacks import EarlyStopping

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


# Load Data

In [3]:
# Menampilkan lima baris pertama dari DataFrame app_reviews_df
app_reviews_df = pd.read_csv('ulasan_aplikasi.csv')
app_reviews_df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,e6b660a4-5978-4918-b13c-dc272ab5af11,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Feel game ini kurang, Tolong perbaiki kontrol ...",1,429,9.4.0,2025-04-22 01:19:44,NaN,NaN,9.4.0
1,5df45bd9-0ee8-41d6-8aae-70e3b9037991,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"saya belum bisa tambahkan bintang, yang saya i...",2,147,9.4.0,2025-04-17 22:19:07,NaN,NaN,9.4.0
2,1286832c-731c-4851-bbd9-bd9762dd8bf0,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"gajelas pas semenjak update, 1. lagi enak main...",2,74,9.4.0,2025-04-20 00:31:54,NaN,NaN,9.4.0
3,5457adb1-370d-4b85-9395-0ab9cdde7ada,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Update yg sekarang kurang memuaskan, banyak Bu...",1,204,9.4.0,2025-04-19 15:19:56,NaN,NaN,9.4.0
4,c0a023e6-0b7c-411e-9891-572fac05f437,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"game nya bagus, Dan satu lagi tolong di perbai...",5,46,9.4.0,2025-04-26 18:26:16,NaN,NaN,9.4.0


# Text Preprocessing

In [4]:
# Menampilkan informasi tentang DataFrame app_reviews_df
app_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80000 entries, 0 to 79999
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   reviewId              80000 non-null  object 
 1   userName              80000 non-null  object 
 2   userImage             80000 non-null  object 
 3   content               80000 non-null  object 
 4   score                 80000 non-null  int64  
 5   thumbsUpCount         80000 non-null  int64  
 6   reviewCreatedVersion  62000 non-null  object 
 7   at                    80000 non-null  object 
 8   replyContent          0 non-null      float64
 9   repliedAt             0 non-null      float64
 10  appVersion            62000 non-null  object 
dtypes: float64(2), int64(2), object(7)
memory usage: 6.7+ MB


In [5]:
clean_df = app_reviews_df[['reviewId','content','score','thumbsUpCount','at']].copy()
print("Duplikat: \n" + str(clean_df[clean_df.duplicated()].sum())+"\n")
print("Missing Value: \n" + str(clean_df[clean_df.isnull()].sum())+"\n")

Duplikat: 
reviewId         0
content          0
score            0
thumbsUpCount    0
at               0
dtype: object

Missing Value: 
reviewId           0
content            0
score            0.0
thumbsUpCount    0.0
at                 0
dtype: object



In [6]:
slangwords = {
    "@": "di",
    "abis": "habis",
    "wtb": "beli",
    "masi": "masih",
    "wts": "jual",
    "wtt": "tukar",
    "bgt": "banget",
    "maks": "maksimal",
    "udh": "sudah",
    "gk": "tidak",
    "g": "tidak",
    "ga": "tidak",
    "nggak": "tidak",
    "gak": "tidak",
    "yg": "yang",
    "klo": "kalau",
    "gede": "besar",
    "lemot": "lambat",
    "gini": "begini",
    "bgini": "begini",
    "gitu": "begitu",
    "bgtu": "begitu",
    "smpe": "sampai",
    "maen": "main",
    "utk": "untuk",
    "dlm": "dalam",
    "tdk": "tidak",
    "ni": "ini",
    "kpn": "kapan",
    "dll": "dan lain-lain",
    "dsb": "dan sebagainya",
    "dst": "dan seterusnya"
}

In [7]:

def cleaningText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # menghapus mention
    text = re.sub(r'#[A-Za-z0-9]+', '', text) # menghapus hashtag
    text = re.sub(r'RT[\s]', '', text) # menghapus RT
    text = re.sub(r"http\S+", '', text) # menghapus link
    text = re.sub(r'[0-9]+', '', text) # menghapus angka
    text = re.sub(r'[^\w\s]', '', text) # menghapus karakter selain huruf dan angka

    text = text.replace('\n', ' ') # mengganti baris baru dengan spasi
    text = text.translate(str.maketrans('', '', string.punctuation)) # menghapus semua tanda baca
    text = text.strip(' ') # menghapus karakter spasi dari kiri dan kanan teks
    return text

def casefoldingText(text): # Mengubah semua karakter dalam teks menjadi huruf kecil
    text = text.lower()
    return text


def fix_slangwords(text):
    words = text.split()
    fixed_words = []

    for word in words:
        if word.lower() in slangwords:
            fixed_words.append(slangwords[word.lower()])
        else:
            fixed_words.append(word)

    fixed_text = ' '.join(fixed_words)
    return fixed_text

def tokenizingText(text): # Memecah atau membagi string, teks menjadi daftar token
    text = word_tokenize(text)
    return text

def filteringText(text): # Menghapus stopwords dalam teks
    listStopwords = set(stopwords.words('indonesian'))
    listStopwords.update(['iya','yaa','gak','nya','na','sih','ku',"di","ga","ya","gaa","loh","kah","woi","woii","woy"])
    filtered = []
    for txt in text:
        if txt not in listStopwords:
            filtered.append(txt)
    text = filtered
    return text

def stemmingText(text): # Mengurangi kata ke bentuk dasarnya yang menghilangkan imbuhan awalan dan akhiran atau ke akar kata
    # Membuat objek stemmer
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()

    # Memecah teks menjadi daftar kata
    words = text.split()

    # Menerapkan stemming pada setiap kata dalam daftar
    stemmed_words = [stemmer.stem(word) for word in words]

    # Menggabungkan kata-kata yang telah distem
    stemmed_text = ' '.join(stemmed_words)

    return stemmed_text


def toSentence(list_words): # Mengubah daftar kata menjadi kalimat
    sentence = ' '.join(word for word in list_words)
    return sentence

In [8]:
clean_df.head(5)

,reviewId,content,score,thumbsUpCount,at
0,e6b660a4-5978-4918-b13c-dc272ab5af11,"Feel game ini kurang, Tolong perbaiki kontrol ...",1,429,2025-04-22 01:19:44
1,5df45bd9-0ee8-41d6-8aae-70e3b9037991,"saya belum bisa tambahkan bintang, yang saya i...",2,147,2025-04-17 22:19:07
2,1286832c-731c-4851-bbd9-bd9762dd8bf0,"gajelas pas semenjak update, 1. lagi enak main...",2,74,2025-04-20 00:31:54
3,5457adb1-370d-4b85-9395-0ab9cdde7ada,"Update yg sekarang kurang memuaskan, banyak Bu...",1,204,2025-04-19 15:19:56
4,c0a023e6-0b7c-411e-9891-572fac05f437,"game nya bagus, Dan satu lagi tolong di perbai...",5,46,2025-04-26 18:26:16


In [9]:
# Membersihkan teks
clean_df['text_clean'] = clean_df['content'].apply(cleaningText)

# Mengubah huruf menjadi huruf kecil
clean_df['text_casefoldingText'] = clean_df['text_clean'].apply(casefoldingText)

# Mengganti slangwords dengan kata baku
clean_df['text_slangwords'] = clean_df['text_casefoldingText'].apply(fix_slangwords)

# Tokenizing teks (memecah jadi kata-kata)
clean_df['text_tokenizingText'] = clean_df['text_slangwords'].apply(tokenizingText)

# Filtering stopwords (menghapus kata umum)
clean_df['text_stopword'] = clean_df['text_tokenizingText'].apply(filteringText)

# Kalau mau hasil akhir, bisa langsung pakai hasil stemming:
clean_df['text_akhir'] = clean_df['text_stopword'].apply(toSentence)

In [10]:
clean_df.head()

,reviewId,content,score,thumbsUpCount,at,text_clean,text_casefoldingText,text_slangwords,text_tokenizingText,text_stopword,text_akhir
0,e6b660a4-5978-4918-b13c-dc272ab5af11,"Feel game ini kurang, Tolong perbaiki kontrol ...",1,429,2025-04-22 01:19:44,Feel game ini kurang Tolong perbaiki kontrol j...,feel game ini kurang tolong perbaiki kontrol j...,feel game ini kurang tolong perbaiki kontrol j...,"[feel, game, ini, kurang, tolong, perbaiki, ko...","[feel, game, tolong, perbaiki, kontrol, joysti...",feel game tolong perbaiki kontrol joystick res...
1,5df45bd9-0ee8-41d6-8aae-70e3b9037991,"saya belum bisa tambahkan bintang, yang saya i...",2,147,2025-04-17 22:19:07,saya belum bisa tambahkan bintang yang saya in...,saya belum bisa tambahkan bintang yang saya in...,saya belum bisa tambahkan bintang yang saya in...,"[saya, belum, bisa, tambahkan, bintang, yang, ...","[tambahkan, bintang, tolong, mode, ekshibisi, ...",tambahkan bintang tolong mode ekshibisi mode o...
2,1286832c-731c-4851-bbd9-bd9762dd8bf0,"gajelas pas semenjak update, 1. lagi enak main...",2,74,2025-04-20 00:31:54,gajelas pas semenjak update lagi enak main ti...,gajelas pas semenjak update lagi enak main ti...,gajelas pas semenjak update lagi enak main tib...,"[gajelas, pas, semenjak, update, lagi, enak, m...","[gajelas, pas, semenjak, update, enak, main, t...",gajelas pas semenjak update enak main tibatiba...
3,5457adb1-370d-4b85-9395-0ab9cdde7ada,"Update yg sekarang kurang memuaskan, banyak Bu...",1,204,2025-04-19 15:19:56,Update yg sekarang kurang memuaskan banyak Bug...,update yg sekarang kurang memuaskan banyak bug...,update yang sekarang kurang memuaskan banyak b...,"[update, yang, sekarang, kurang, memuaskan, ba...","[update, memuaskan, bug, bug, jaringan, kali, ...",update memuaskan bug bug jaringan kali dirugik...
4,c0a023e6-0b7c-411e-9891-572fac05f437,"game nya bagus, Dan satu lagi tolong di perbai...",5,46,2025-04-26 18:26:16,game nya bagus Dan satu lagi tolong di perbaik...,game nya bagus dan satu lagi tolong di perbaik...,game nya bagus dan satu lagi tolong di perbaik...,"[game, nya, bagus, dan, satu, lagi, tolong, di...","[game, bagus, tolong, perbaiki, kasih, umpan, ...",game bagus tolong perbaiki kasih umpan lambung...


# Pelabelan Data

In [11]:
# Membaca data kamus kata-kata positif dari GitHub
lexicon_positive = dict()

response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_positive.csv')
# Mengirim permintaan HTTP untuk mendapatkan file CSV dari GitHub

if response.status_code == 200:
    # Jika permintaan berhasil
    reader = csv.reader(StringIO(response.text), delimiter=',')
    # Membaca teks respons sebagai file CSV menggunakan pembaca CSV dengan pemisah koma

    for row in reader:
        # Mengulangi setiap baris dalam file CSV
        lexicon_positive[row[0]] = int(row[1])
        # Menambahkan kata-kata positif dan skornya ke dalam kamus lexicon_positive
else:
    print("Failed to fetch positive lexicon data")

# Membaca data kamus kata-kata negatif dari GitHub
lexicon_negative = dict()

response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_negative.csv')
# Mengirim permintaan HTTP untuk mendapatkan file CSV dari GitHub

if response.status_code == 200:
    # Jika permintaan berhasil
    reader = csv.reader(StringIO(response.text), delimiter=',')
    # Membaca teks respons sebagai file CSV menggunakan pembaca CSV dengan pemisah koma

    for row in reader:
        # Mengulangi setiap baris dalam file CSV
        lexicon_negative[row[0]] = int(row[1])
        # Menambahkan kata-kata negatif dan skornya dalam kamus lexicon_negative
else:
    print("Failed to fetch negative lexicon data")

In [12]:
# Fungsi untuk menentukan polaritas sentimen dari tweet
def sentiment_analysis_lexicon_indonesia(text):
    #for word in text:

    score = 0
    # Inisialisasi skor sentimen ke 0

    for word in text:
        # Mengulangi setiap kata dalam teks

        if (word in lexicon_positive):
            score = score + lexicon_positive[word]
            # Jika kata ada dalam kamus positif, tambahkan skornya ke skor sentimen

    for word in text:
        # Mengulangi setiap kata dalam teks (sekali lagi)

        if (word in lexicon_negative):
            score = score + lexicon_negative[word]
            # Jika kata ada dalam kamus negatif, kurangkan skornya dari skor sentimen

    polarity=''
    # Inisialisasi variabel polaritas

    if (score >= 0):
        polarity = 'positive'
        # Jika skor sentimen lebih besar atau sama dengan 0, maka polaritas adalah positif
    elif (score < 0):
        polarity = 'negative'
        # Jika skor sentimen kurang dari 0, maka polaritas adalah negatif

    # else:
    #     polarity = 'neutral'
    # Ini adalah bagian yang bisa digunakan untuk menentukan polaritas netral jika diperlukan

    return score, polarity
    # Mengembalikan skor sentimen dan polaritas teks

In [13]:
results = clean_df['text_stopword'].apply(sentiment_analysis_lexicon_indonesia)
results = list(zip(*results))
clean_df['polarity_score'] = results[0]
clean_df['polarity'] = results[1]
print(clean_df['polarity'].value_counts())

polarity
negative    47585
positive    32415
Name: count, dtype: int64


In [14]:
# Pisahkan data menjadi fitur (tweet) dan label (sentimen)
X = clean_df['text_akhir']
y = clean_df['polarity']

# Ekstraksi Fitur dan Splitting Data

## Bag of Words

In [15]:
def bow(X,y, test_size):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(X)

    # Bagi data menjadi data latih dan data uji
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
    return X_train, X_test, y_train, y_test, vectorizer

### Data Splitting 80-20

In [16]:
x_train1, x_test1, y_train1, y_test1, bow1 = bow(X,y,0.2)

### Data Splitting 75-25

In [17]:
x_train2, x_test2, y_train2, y_test2, bow2 = bow(X,y,0.25)

### Data Splitting 70-30

In [18]:
x_train3, x_test3, y_train3, y_test3, bow3 = bow(X,y,0.3)

# Perancangan Model ML

## Skema 1: LinearSVC / BOW / 80-20

In [19]:
svc = LinearSVC(max_iter=2000, random_state=42)

# Melatih model pada data pelatihan
svc.fit(x_train1, y_train1)

# Prediksi sentimen pada data pelatihan dan data uji
y_pred_train = svc.predict(x_train1)
y_pred_test = svc.predict(x_test1)


print(svc.__class__.__name__, '/ BOW / 80-20')

# Evaluasi akurasi model pada data pelatihan
accuracy_train = accuracy_score(y_pred_train, y_train1)
print("Akurasi training: ", accuracy_train)

# Evaluasi akurasi model pada data uji
accuracy_test = accuracy_score(y_pred_test, y_test1)
print("Akurasi testing: ", accuracy_test)

LinearSVC / BOW / 80-20
Akurasi training:  0.999375
Akurasi testing:  0.959125


## Skema 2: Logistic Regression / BOW / 75-25

In [20]:
lr = LogisticRegression(max_iter=2000, random_state=42)

# Melatih model pada data pelatihan
lr.fit(x_train2, y_train2)

# Prediksi sentimen pada data pelatihan dan data uji
y_pred_train = lr.predict(x_train2)
y_pred_test = lr.predict(x_test2)


print(lr.__class__.__name__, '/ BOW / 75-25')

# Evaluasi akurasi model pada data pelatihan
accuracy_train = accuracy_score(y_pred_train, y_train2)
print("Akurasi training: ", accuracy_train)

# Evaluasi akurasi model pada data uji
accuracy_test = accuracy_score(y_pred_test, y_test2)
print("Akurasi testing: ", accuracy_test)

LogisticRegression / BOW / 75-25
Akurasi training:  0.98765
Akurasi testing:  0.9514


## Skema 3: XGB / BOW / 70-30

In [21]:
encoded_y_train3 = y_train3.map(lambda x: 1 if x == 'positive' else 0)
encoded_y_test3 = y_test3.map(lambda x: 1 if x == 'positive' else 0)

In [22]:
xgb = XGBClassifier()

# Melatih model pada data pelatihan
xgb.fit(x_train3, encoded_y_train3)

# Prediksi sentimen pada data pelatihan dan data uji
y_pred_train = xgb.predict(x_train3)
y_pred_test = xgb.predict(x_test3)


print(xgb.__class__.__name__, '/ BOW / 70-30')

# Evaluasi akurasi model pada data pelatihan
accuracy_train = accuracy_score(y_pred_train, encoded_y_train3)
print("Akurasi training: ", accuracy_train)

# Evaluasi akurasi model pada data uji
accuracy_test = accuracy_score(y_pred_test, encoded_y_test3)
print("Akurasi testing: ", accuracy_test)

XGBClassifier / BOW / 70-30
Akurasi training:  0.8876607142857142
Akurasi testing:  0.856875


# Perancangan Model Deep Learning

## Skema 4: DNN / Word Embedding / 80-20

### Text Preprocessing

In [23]:
# Tokenisasi
tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')  # batasi vocab
tokenizer.fit_on_texts(X)

In [24]:
# Padding
sekuens = tokenizer.texts_to_sequences(X)
panjang_ulasan = [len(x) for x in sekuens]
maxlen = int(np.percentile(panjang_ulasan, 90))
padded = pad_sequences(sekuens, maxlen=maxlen, padding='post', truncating='post')

In [25]:
# Mengubah label sentimen menjadi angka
labels = clean_df['polarity'].map({'positive': 1, 'negative': 0}).values

# Pembagian Data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(padded, labels, test_size=0.2, random_state=42)

### Kerangka Model

In [26]:
model = Sequential([
    Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=64),
    Flatten(),
    Dense(8, activation='relu'),
    Dense(8, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

### Compile dan Training Model

In [27]:
from tensorflow.keras.callbacks import EarlyStopping
# Stop training kalau val_loss ga membaik selama 3 epoch
earlystop = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True,
    verbose=1
)

In [28]:
callbacks = [earlystop]

In [29]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [30]:
history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_test, y_test),
    callbacks=callbacks
)

Epoch 1/10
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.7752 - loss: 0.4319 - val_accuracy: 0.8884 - val_loss: 0.2453
Epoch 2/10
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9449 - loss: 0.1393 - val_accuracy: 0.9189 - val_loss: 0.2153
Epoch 3/10
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.9775 - loss: 0.0645 - val_accuracy: 0.9193 - val_loss: 0.2388
Epoch 4/10
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9886 - loss: 0.0338 - val_accuracy: 0.9160 - val_loss: 0.2872
Epoch 5/10
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.9939 - loss: 0.0199 - val_accuracy: 0.9149 - val_loss: 0.3417
Epoch 5: early stopping
Restoring model weights from the end of the best epoch: 2.


### Evaluasi Model

In [31]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.2f}")

500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9157 - loss: 0.2169
Test Accuracy: 0.92


# Inference Cell

In [32]:
def hasil_sentimen_dl(doc):
    new_sequences = tokenizer.texts_to_sequences(doc)
    padded_new_sequences = pad_sequences(new_sequences, maxlen=maxlen, padding='post', truncating='post')

    predictions = model.predict(padded_new_sequences)

    labels = []
    for pred in predictions:
        if pred > 0.5:
            labels.append('positive')
        else:
            labels.append('negative')
    return np.array(labels)

In [33]:
def hasil_sentimen_ml(model, vectorizer, documents):
    """Memprediksi sentimen dari list dokumen menggunakan model ML."""
    vectorized_documents = vectorizer.transform(documents)
    predictions_numeric = model.predict(vectorized_documents)
    predicted_labels = []
    for pred in predictions_numeric:
        if model == xgb:
            if pred == 1:
                predicted_labels.append('positive')
            else:
                predicted_labels.append('negative')
        else:
            predicted_labels.append(pred)
    return np.array(predicted_labels)

In [34]:
clean_df[clean_df['thumbsUpCount'] > 9000]

,reviewId,content,score,thumbsUpCount,at,text_clean,text_casefoldingText,text_slangwords,text_tokenizingText,text_stopword,text_akhir,polarity_score,polarity
6779,81f966c1-0284-4a49-9b09-7b121196c530,"Tolong untuk developer game pes, agar tampilan...",4,34380,2022-03-30 16:41:58,Tolong untuk developer game pes agar tampilan ...,tolong untuk developer game pes agar tampilan ...,tolong untuk developer game pes agar tampilan ...,"[tolong, untuk, developer, game, pes, agar, ta...","[tolong, developer, game, pes, tampilan, stik,...",tolong developer game pes tampilan stik arah t...,-8,negative
7557,069e00f5-f00b-4d31-a201-533714257f41,"Game nya seru banget, grafik oke..kontrol bola...",5,9089,2022-05-15 01:21:15,Game nya seru banget grafik okekontrol bola ny...,game nya seru banget grafik okekontrol bola ny...,game nya seru banget grafik okekontrol bola ny...,"[game, nya, seru, banget, grafik, okekontrol, ...","[game, seru, banget, grafik, okekontrol, bola,...",game seru banget grafik okekontrol bola lumaya...,-1,negative
7712,5d41f17b-8b9f-4fbe-b554-c024d07ec109,Saya sarankan untuk pembaruan sistem segera di...,1,16700,2022-04-01 21:45:42,Saya sarankan untuk pembaruan sistem segera di...,saya sarankan untuk pembaruan sistem segera di...,saya sarankan untuk pembaruan sistem segera di...,"[saya, sarankan, untuk, pembaruan, sistem, seg...","[sarankan, pembaruan, sistem, susah, menemukan...",sarankan pembaruan sistem susah menemukan lawa...,-16,negative
8200,4b1954ca-9297-4a3c-af8f-3fbe00865bab,"game oke, cuma tolong! itu wasitnya di bikin i...",5,11281,2024-07-18 20:36:49,game oke cuma tolong itu wasitnya di bikin iku...,game oke cuma tolong itu wasitnya di bikin iku...,game oke cuma tolong itu wasitnya di bikin iku...,"[game, oke, cuma, tolong, itu, wasitnya, di, b...","[game, oke, tolong, wasitnya, bikin, lari, lap...",game oke tolong wasitnya bikin lari lapangan t...,6,positive
62001,6b70c61e-36ff-4faf-9cc3-bbd717169117,"Game sampah!!! Gameplay susah dimainkan, mau n...",1,20426,2023-09-28 08:10:53,Game sampah Gameplay susah dimainkan mau ngope...,game sampah gameplay susah dimainkan mau ngope...,game sampah gameplay susah dimainkan mau ngope...,"[game, sampah, gameplay, susah, dimainkan, mau...","[game, sampah, gameplay, susah, dimainkan, ngo...",game sampah gameplay susah dimainkan ngoper sa...,-18,negative


In [35]:
doc = [
    'Tolong untuk developer game pes, agar tampilan stik arah "tetap"diperbaiki. saat bermain jangan berhenti di titik itu aja. soalnya kami kesulitan dan terkadang tidak tersentuh oleh jari kita, semoga untuk tampilan stik arah "tetap" diperbaiki agar saat digeser, posisi nya berubah sesuai letak jari kita namun titik letaknya stik berhenti di titik tersebut.',
    'Game sampah!!! Gameplay susah dimainkan, mau ngoper salah analog ntah kemana mana. Pemain muda lelah, kontrak pemain makin dipersulit. eFootball game sampah!!',
    'Saya sarankan untuk pembaruan sistem segera dikarenakan, sangat susah sekali menemukan lawan di event. Tolong, segera perbaharui sistem, saya sedikit kesal menunggu musuh lama lama malah keluar sendiri, dan pas match lagi malah trafic padat. Serupa pula dengan, saat akan berlaga dengan teman, trafic padat terus saja ada.',
    'game oke, cuma tolong! itu wasitnya di bikin ikut lari di lapangan, juga tambahkan mode cuaca hujan salju, untuk liga tambahkan top scor, assist, dll . dan tolong perbaiki untuk tombol umpan trobosan terkadang bukan umpan malah tembak. untuk sleding manual juga gak akurat/delay. dan satu lagi tambahkan mode laga uji coba kemampuan, bebas memilih lawan vs com maupun player. itu aja terima kasih',
    'Game nya seru banget, grafik oke..kontrol bola nya juga lumayan keren...semoga ke depan nya lebih baik lagi dan di kembangkan lagi game nya...oh iya 1 yg bikin kecewa, knp match nya susah banget untuk masuk yah? padahal untuk menaikan peringkat...kl gk di maenin match nya, masa peringkat dasar-dasar terus....'
]

In [36]:
print("SVC")
for i,label in enumerate(hasil_sentimen_ml(svc, bow1, doc)):
    print(f"Dokumen {i+1}: {label}")
print("\n\n")
print("Logistic Regression")
for i,label in enumerate(hasil_sentimen_ml(lr, bow2, doc)):
    print(f"Dokumen {i+1}: {label}")
print("\n\n")
print("XGBoost")
for i,label in enumerate(hasil_sentimen_ml(xgb, bow3, doc)):
    print(f"Dokumen {i+1}: {label}")
print("\n\n")
print("Deep Neural Network")
for i,label in enumerate(hasil_sentimen_dl(doc)):
    print(f"Dokumen {i+1}: {label}")

SVC
Dokumen 1: negative
Dokumen 2: negative
Dokumen 3: negative
Dokumen 4: positive
Dokumen 5: positive



Logistic Regression
Dokumen 1: negative
Dokumen 2: negative
Dokumen 3: negative
Dokumen 4: positive
Dokumen 5: positive



XGBoost
Dokumen 1: negative
Dokumen 2: negative
Dokumen 3: negative
Dokumen 4: positive
Dokumen 5: negative



Deep Neural Network
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 326ms/step
Dokumen 1: negative
Dokumen 2: negative
Dokumen 3: negative
Dokumen 4: positive
Dokumen 5: positive
